In [21]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define currency pairs
currency_pairs = [
    'EURUSD=X', 'JPYUSD=X', 'GBPUSD=X', 'AUDUSD=X', 'CADUSD=X',
    'CHFUSD=X', 'CNYUSD=X', 'INRUSD=X', 'BRLUSD=X', 'ZARUSD=X'
]

# Download data
data = yf.download(currency_pairs, start='2015-01-01', end='2023-01-01')['Close']

# Rename columns (remove '=X' for clarity)
data.columns = [col.replace('=X', '') for col in data.columns]

# Compute log returns
returns = np.log(data / data.shift(1)).dropna()

# Show first few rows
print(returns.head())


/var/folders/sx/9mnbjp0s38q6c18pfnj9ybs40000gn/T/ipykernel_26986/1896519675.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(currency_pairs, start='2015-01-01', end='2023-01-01')['Close']
[*********************100%***********************]  10 of 10 completed


              AUDUSD    BRLUSD    CADUSD    CHFUSD    CNYUSD    EURUSD  \
Date                                                                     
2015-01-02  0.000817  0.000377  0.000052  0.000010  0.000000 -0.000762   
2015-01-05 -0.013565 -0.014221 -0.015295 -0.011701  0.000000 -0.011897   
2015-01-06  0.003961 -0.004412  0.003057 -0.000398 -0.001983 -0.000621   
2015-01-07 -0.002427  0.002408 -0.006781 -0.005056  0.001031 -0.005346   
2015-01-08 -0.000565  0.005876  0.001564 -0.003396 -0.000322 -0.003320   

              GBPUSD    INRUSD    JPYUSD    ZARUSD  
Date                                                
2015-01-02 -0.000078  0.000000 -0.001645 -0.008990  
2015-01-05 -0.019104 -0.003959 -0.004694 -0.014974  
2015-01-06 -0.001741 -0.000948  0.008413  0.002767  
2015-01-07 -0.007918 -0.003625  0.006317 -0.002110  
2015-01-08 -0.002011  0.004651 -0.005177  0.002777  


In [22]:
# Assuming 'returns' is your DataFrame from earlier

# Mean return vector (mu) - use .mean()
mu = returns.mean()

# Covariance matrix (Sigma) - use .cov()
cov_matrix = returns.cov()

# Display results
print("Mean Return Vector (μ):")
print(mu)

print("\nCovariance Matrix (Σ):")
print(cov_matrix)


Mean Return Vector (μ):
AUDUSD   -0.000089
BRLUSD   -0.000325
CADUSD   -0.000073
CHFUSD    0.000038
CNYUSD   -0.000056
EURUSD   -0.000059
GBPUSD   -0.000121
INRUSD   -0.000129
JPYUSD   -0.000049
ZARUSD   -0.000186
dtype: float64

Covariance Matrix (Σ):
          AUDUSD    BRLUSD    CADUSD    CHFUSD    CNYUSD    EURUSD    GBPUSD  \
AUDUSD  0.000042  0.000026  0.000021  0.000014  0.000006  0.000017  0.000022   
BRLUSD  0.000026  0.000131  0.000018  0.000008  0.000004  0.000012  0.000015   
CADUSD  0.000021  0.000018  0.000024  0.000007  0.000003  0.000010  0.000014   
CHFUSD  0.000014  0.000008  0.000007  0.000039  0.000003  0.000018  0.000013   
CNYUSD  0.000006  0.000004  0.000003  0.000003  0.000007  0.000004  0.000005   
EURUSD  0.000017  0.000012  0.000010  0.000018  0.000004  0.000026  0.000018   
GBPUSD  0.000022  0.000015  0.000014  0.000013  0.000005  0.000018  0.000038   
INRUSD  0.000008  0.000012  0.000006  0.000003  0.000003  0.000004  0.000006   
JPYUSD  0.000008  0.000006 

In [23]:
import cvxpy as cp
import numpy as np
import pandas as pd

# Convert data to numpy
mu_vector = mu.values
sigma_matrix = cov_matrix.values
n = len(mu_vector)

# Define optimization variable
w = cp.Variable(n)

# Objective: minimize portfolio variance
objective = cp.Minimize(cp.quad_form(w, sigma_matrix))

# Constraint: fully invested (weights sum to 1)
constraints = [cp.sum(w) == 1]

# Solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Get weights
weights_mvp = w.value

# Display weights as pandas Series
weights_series = pd.Series(weights_mvp, index=returns.columns)
print("Minimum Variance Portfolio Weights:")
print(weights_series)



Minimum Variance Portfolio Weights:
AUDUSD   -0.075910
BRLUSD    0.001430
CADUSD    0.164580
CHFUSD    0.017401
CNYUSD    0.576245
EURUSD    0.033591
GBPUSD   -0.002284
INRUSD    0.232218
JPYUSD    0.108111
ZARUSD   -0.055382
dtype: float64


In [24]:
import numpy as np
import pandas as pd
import cvxpy as cp

mu_vector = mu.values
sigma_matrix = cov_matrix.values
n = len(mu_vector)

# Risk aversion parameters grid (lambda)
lambdas = np.logspace(-3, 3, 100)

best_sharpe = -np.inf
best_weights = None

for lam in lambdas:
    w = cp.Variable(n)
    ret = mu_vector @ w
    risk = cp.quad_form(w, sigma_matrix)
    
    objective = cp.Maximize(ret - lam * risk)
    constraints = [cp.sum(w) == 1,w <= 5]
    problem = cp.Problem(objective, constraints)
    
    problem.solve()
    
    if w.value is None:
        continue
    
    weights = w.value
    port_return = mu_vector @ weights
    port_risk = np.sqrt(weights.T @ sigma_matrix @ weights)
    
    sharpe = port_return / port_risk if port_risk > 0 else 0
    
    if sharpe > best_sharpe:
        best_sharpe = sharpe
        best_weights = weights

print("Max Sharpe Ratio Portfolio Weights (approx):")
print(pd.Series(best_weights, index=returns.columns))
print(f"Sharpe Ratio: {best_sharpe:.4f}")



Max Sharpe Ratio Portfolio Weights (approx):
AUDUSD    0.607434
BRLUSD   -2.700512
CADUSD    3.051034
CHFUSD    5.000000
CNYUSD    4.304623
EURUSD   -0.224484
GBPUSD   -3.198054
INRUSD   -4.321398
JPYUSD   -0.743331
ZARUSD   -0.775312
dtype: float64
Sharpe Ratio: 0.0342
